In [27]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

from ISLP.models import (ModelSpec as MS,
                        summarize,
                        poly)

In [18]:
auto = pd.read_csv('../datasets/Auto.csv', na_values=['?'])
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [22]:
# drop nans
auto = auto.dropna()

In [26]:
y = auto['mpg']
X = pd.DataFrame({
    'intercept': np.ones(auto.shape[0]),
    'horsepower': auto['horsepower']
})

model = sm.OLS(y, X)

# Fit the model
results = model.fit()

In [28]:
summarize(results)

,coef,std err,t,P>|t|
intercept,39.9359,0.717,55.660,0.0
horsepower,-0.1578,0.006,-24.489,0.0
